In [1]:
library(tidyverse)

library(ggrepel)
library(DOSE)
library(org.Mm.eg.db)
library(clusterProfiler)
library(enrichplot)
library(msigdbr)

library(GO.db)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


DOSE v3.28.1  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use DOSE in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Guang-Rong Yan, Qing-Yu He. DOSE: an R/Bioconductor package for Disease Ontology Semantic and Enrichment analysis. Bioinformatics 2015, 31(4):608-609


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The foll

In [2]:
setwd('~/stereoseq/20231128-HD')
deg_df <- read.csv('./DEG.csv', sep='\t')

print(length(rownames(deg_df)))

[1] 114716


In [3]:
cluster_df <- deg_df[deg_df$Cluster == '0', ]
deseq_results <- cluster_df[order(cluster_df$Log2FoldChange, decreasing = TRUE), ]
geneList <- deseq_results$Log2FoldChange
names(geneList) <- deseq_results$Gene
print(tail(geneList))


  Spdye4c      Tal2       Mt1     Tinag       Mt2       Vim 
-21.24925 -21.44939 -21.83168 -22.03260 -25.62024 -40.62363 


In [4]:
gsea_results <- gseGO(
  geneList = geneList,
  OrgDb = org.Mm.eg.db,
  keyType = "SYMBOL",
  ont = "ALL",
  minGSSize = 2,
  maxGSSize = 1000,
  pvalueCutoff = 1,
  verbose = FALSE
)

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”


In [5]:
p <- dotplot(gsea_results, showCategory=10, split=".sign") + facet_grid(.~.sign)
ggsave('./gsea_results.png', p, width=10, height=10)

In [6]:
for(i in unique(deg_df$Cluster)){
  cluster_df <- deg_df[deg_df$Cluster == i, ]
  deseq_results <- cluster_df[order(cluster_df$Log2FoldChange, decreasing = TRUE), ]
  geneList <- deseq_results$Log2FoldChange
  names(geneList) <- deseq_results$Gene

  gsea_results <- gseGO(
    geneList = geneList,
    OrgDb = org.Mm.eg.db,
    keyType = "SYMBOL",
    ont = "ALL",
    minGSSize = 2,
    maxGSSize = 1000,
    pvalueCutoff = 1,
    verbose = FALSE
  )
#   p <- dotplot(gsea_results, showCategory=10, split=".sign") + facet_grid(.~.sign)
#   ggsave(paste0('./gsea_results_', i, '.png'), p, width=10, height=10)
  write.csv(gsea_results@result, paste0('./gsea_results_', i, '.csv'))
}

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“There were 184 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.01% of the list).
The order of those 